In [48]:
import cv2
import numpy as np
import os
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
from PIL import Image

The OS module in Python provides a way of using operating system dependent functionality.
Python method listdir() returns a list containing the names of the entries in the directory given by path.
os.chdir() method in Python used to change the current working directory to specified path.

In [49]:
print(os.getcwd()) 

C:\Users\tetal\AI\Test001


In [33]:
# Folder which contains all the images 
# from which video is to be generated 
os.chdir("C:\\Users\\tetal\\AI\\Test001") 
path = "C:\\Users\\tetal\\AI\\Test001"

In [34]:
mean_height = 0
mean_width = 0

num_of_images = len(os.listdir('.')) 
print(num_of_images) 

201


In [35]:
	image_folder = 'C:\\Users\\tetal\\AI\\Test001' # make sure to use your folder 
	video_name = 'mygeneratedvideo2.mp4'
	os.chdir("C:\\Users\\tetal\\AI") 
images = [img for img in os.listdir(image_folder) 
			if img.endswith(".tif") or
				img.endswith(".jpeg") or
				img.endswith("png")] 
print(images)

['001.tif', '002.tif', '003.tif', '004.tif', '005.tif', '006.tif', '007.tif', '008.tif', '009.tif', '010.tif', '011.tif', '012.tif', '013.tif', '014.tif', '015.tif', '016.tif', '017.tif', '018.tif', '019.tif', '020.tif', '021.tif', '022.tif', '023.tif', '024.tif', '025.tif', '026.tif', '027.tif', '028.tif', '029.tif', '030.tif', '031.tif', '032.tif', '033.tif', '034.tif', '035.tif', '036.tif', '037.tif', '038.tif', '039.tif', '040.tif', '041.tif', '042.tif', '043.tif', '044.tif', '045.tif', '046.tif', '047.tif', '048.tif', '049.tif', '050.tif', '051.tif', '052.tif', '053.tif', '054.tif', '055.tif', '056.tif', '057.tif', '058.tif', '059.tif', '060.tif', '061.tif', '062.tif', '063.tif', '064.tif', '065.tif', '066.tif', '067.tif', '068.tif', '069.tif', '070.tif', '071.tif', '072.tif', '073.tif', '074.tif', '075.tif', '076.tif', '077.tif', '078.tif', '079.tif', '080.tif', '081.tif', '082.tif', '083.tif', '084.tif', '085.tif', '086.tif', '087.tif', '088.tif', '089.tif', '090.tif', '091.tif'

In [36]:
# Folder which contains all the images 
# from which video is to be generated 
os.chdir("C:\\Users\\tetal\\AI\\Test001") 
path = "C:\\Users\\tetal\\AI\\Test001"
mean_height = 0
mean_width = 0

num_of_images = len(os.listdir('.')) 
print(num_of_images) 

201


In [38]:
for file in os.listdir('.'): 
	im = Image.open(os.path.join(path, file)) 
	width, height = im.size 
	mean_width += width 
	mean_height += height 
	# im.show() # uncomment this for displaying the image 

# Finding the mean height and width of all images. 
# This is required because the video frame needs 
# to be set with same width and height. Otherwise 
# images not equal to that width height will not get 
# embedded into the video 
mean_width = int(mean_width / num_of_images) 
mean_height = int(mean_height / num_of_images) 

# print(mean_height) 
# print(mean_width) 

# Resizing of the images to give 
# them same width and height 
for file in os.listdir('.'): 
	if file.endswith(".tif") or file.endswith(".jpeg") or file.endswith("png"): 
		# opening image using PIL Image 
		im = Image.open(os.path.join(path, file)) 

		# im.size includes the height and width of image 
		width, height = im.size 
		print(width, height) 

		# resizing 
		imResize = im.resize((238, 158), Image.ANTIALIAS) 
		imResize.save( file, "JPEG",quality = 95) # setting quality 
		# printing each resized image name 
		print(im.filename.split('\\')[-1], " is resized") 


238 158
001.tif  is resized
238 158
002.tif  is resized
238 158
003.tif  is resized
238 158
004.tif  is resized
238 158
005.tif  is resized
238 158
006.tif  is resized
238 158
007.tif  is resized
238 158
008.tif  is resized
238 158
009.tif  is resized
238 158
010.tif  is resized
238 158
011.tif  is resized
238 158
012.tif  is resized
238 158
013.tif  is resized
238 158
014.tif  is resized
238 158
015.tif  is resized
238 158
016.tif  is resized
238 158
017.tif  is resized
238 158
018.tif  is resized
238 158
019.tif  is resized
238 158
020.tif  is resized
238 158
021.tif  is resized
238 158
022.tif  is resized
238 158
023.tif  is resized
238 158
024.tif  is resized
238 158
025.tif  is resized
238 158
026.tif  is resized
238 158
027.tif  is resized
238 158
028.tif  is resized
238 158
029.tif  is resized
238 158
030.tif  is resized
238 158
031.tif  is resized
238 158
032.tif  is resized
238 158
033.tif  is resized
238 158
034.tif  is resized
238 158
035.tif  is resized
238 158
036.tif  is 

In [39]:
# Video Generating function 
def generate_video(images,outvid=None, fps=10, size=None,
               is_color=True, format="XVID"): 
	#image_folder = 'C:\\Users\\tetal\\AI\\Test001' # make sure to use your folder 
	video_name = 'mygeneratedvideo2.mp4'
	#os.chdir("C:\\Users\\tetal\\AI\\Test001") 
	
	#images = [img for img in os.listdir(image_folder) 
			#if img.endswith(".tif") or
				#img.endswith(".jpeg") or
				#img.endswith("png")] 
	
	# Array images should only consider 
	# the image files ignoring others if any 
	print(images) 

	frame = cv2.imread(os.path.join(image_folder, images[0])) 

	# setting the frame width, height width 
	# the width, height of first image 
	height, width, layers = frame.shape 
	fourcc = VideoWriter_fourcc(*format)
	video = VideoWriter(video_name, fourcc, float(fps), (238, 158), is_color)

	# Appending the images to the video one by one 
	for image in images: 
		video.write(cv2.imread(os.path.join(image_folder, image))) 
	
	# Deallocating memories taken for window creation 
	cv2.destroyAllWindows() 
    
	video.release() # releasing the video generated 




In [40]:
# Calling the generate_video function 
generate_video(images) 

['001.tif', '002.tif', '003.tif', '004.tif', '005.tif', '006.tif', '007.tif', '008.tif', '009.tif', '010.tif', '011.tif', '012.tif', '013.tif', '014.tif', '015.tif', '016.tif', '017.tif', '018.tif', '019.tif', '020.tif', '021.tif', '022.tif', '023.tif', '024.tif', '025.tif', '026.tif', '027.tif', '028.tif', '029.tif', '030.tif', '031.tif', '032.tif', '033.tif', '034.tif', '035.tif', '036.tif', '037.tif', '038.tif', '039.tif', '040.tif', '041.tif', '042.tif', '043.tif', '044.tif', '045.tif', '046.tif', '047.tif', '048.tif', '049.tif', '050.tif', '051.tif', '052.tif', '053.tif', '054.tif', '055.tif', '056.tif', '057.tif', '058.tif', '059.tif', '060.tif', '061.tif', '062.tif', '063.tif', '064.tif', '065.tif', '066.tif', '067.tif', '068.tif', '069.tif', '070.tif', '071.tif', '072.tif', '073.tif', '074.tif', '075.tif', '076.tif', '077.tif', '078.tif', '079.tif', '080.tif', '081.tif', '082.tif', '083.tif', '084.tif', '085.tif', '086.tif', '087.tif', '088.tif', '089.tif', '090.tif', '091.tif'

In [45]:
cap = cv2.VideoCapture('mygeneratedvideo2.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

count=0
index = 0;
ret,frame = cap.read()
while(ret):
   
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    ret,frame = cap.read()
    
cv2.destroyAllWindows()
cap.release()

In [42]:

cap = cv2.VideoCapture("mygeneratedvideo2.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
count=0
index = 0;
ret, frame2 = cap.read()
while(ret):
   
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    #print(hsv)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    if count==10:
        count=0
    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):    
        prvs = next
    ret, frame2 = cap.read()

cap.release()
cv2.destroyAllWindows()

[[[ 63 255   0]
  [  7 255   2]
  [178 255   4]
  ...
  [121 255  25]
  [122 255  25]
  [121 255  27]]

 [[ 29 255   3]
  [ 11 255   5]
  [  4 255   6]
  ...
  [118 255   7]
  [121 255   3]
  [120 255   3]]

 [[ 25 255   2]
  [  6 255   5]
  [179 255   6]
  ...
  [110 255   0]
  [ 27 255   6]
  [ 27 255   8]]

 ...

 [[ 87 255   3]
  [ 85 255   5]
  [ 84 255   6]
  ...
  [171 255   4]
  [170 255   5]
  [169 255   6]]

 [[ 93 255   3]
  [ 91 255   4]
  [ 90 255   5]
  ...
  [176 255   3]
  [174 255   4]
  [173 255   4]]

 [[ 94 255   3]
  [ 92 255   4]
  [ 91 255   4]
  ...
  [  5 255   3]
  [  1 255   3]
  [179 255   3]]]
[[[172 255   4]
  [168 255   6]
  [163 255   8]
  ...
  [121 255  12]
  [128 255   4]
  [134 255   3]]

 [[179 255   5]
  [171 255   7]
  [165 255   9]
  ...
  [121 255  11]
  [160 255   1]
  [ 13 255   3]]

 [[175 255   5]
  [167 255   7]
  [162 255   9]
  ...
  [123 255   9]
  [  8 255   4]
  [ 16 255   7]]

 ...

 [[ 91 255   1]
  [ 89 255   2]
  [ 87 255   2]
  ..

[[[ 32 255   8]
  [ 31 255   8]
  [ 32 255   7]
  ...
  [160 255   9]
  [162 255   9]
  [177 255   7]]

 [[ 38 255   7]
  [ 37 255   7]
  [ 37 255   6]
  ...
  [154 255  10]
  [164 255  10]
  [ 16 255  15]]

 [[ 44 255   6]
  [ 43 255   5]
  [ 43 255   4]
  ...
  [152 255   9]
  [  9 255  15]
  [ 22 255  26]]

 ...

 [[179 255  41]
  [179 255  43]
  [179 255  44]
  ...
  [ 59 255   0]
  [ 58 255   0]
  [ 60 255   0]]

 [[179 255  33]
  [179 255  34]
  [178 255  35]
  ...
  [ 38 255   1]
  [ 40 255   1]
  [ 45 255   0]]

 [[179 255  27]
  [178 255  28]
  [178 255  29]
  ...
  [ 32 255   0]
  [ 33 255   0]
  [ 36 255   0]]]
[[[ 48 255   3]
  [ 45 255   3]
  [ 47 255   3]
  ...
  [172 255  10]
  [170 255   9]
  [  3 255   8]]

 [[ 53 255   4]
  [ 52 255   3]
  [ 56 255   2]
  ...
  [170 255  10]
  [177 255  11]
  [ 17 255  16]]

 [[ 60 255   4]
  [ 61 255   3]
  [ 65 255   2]
  ...
  [174 255  10]
  [ 15 255  19]
  [ 22 255  27]]

 ...

 [[  3 255  52]
  [  3 255  53]
  [  2 255  53]
  ..

[[[ 25 255   2]
  [ 20 255   2]
  [ 15 255   1]
  ...
  [ 15 255  17]
  [ 13 255  13]
  [ 19 255  13]]

 [[ 46 255   1]
  [ 28 255   0]
  [176 255   0]
  ...
  [ 18 255  24]
  [ 20 255  23]
  [ 23 255  21]]

 [[ 81 255   1]
  [ 97 255   0]
  [118 255   1]
  ...
  [ 21 255  36]
  [ 23 255  32]
  [ 25 255  25]]

 ...

 [[  1 255   9]
  [  1 255   9]
  [  4 255  10]
  ...
  [143 255   3]
  [141 255   3]
  [140 255   2]]

 [[  2 255   8]
  [  1 255   8]
  [  1 255   8]
  ...
  [143 255   3]
  [140 255   2]
  [139 255   2]]

 [[  2 255   7]
  [  1 255   7]
  [  0 255   6]
  ...
  [143 255   2]
  [140 255   1]
  [138 255   1]]]
[[[ 31 255   2]
  [ 24 255   2]
  [ 19 255   1]
  ...
  [ 12 255  17]
  [ 11 255  14]
  [ 17 255  15]]

 [[ 47 255   1]
  [ 35 255   0]
  [ 16 255   0]
  ...
  [ 16 255  24]
  [ 18 255  23]
  [ 21 255  22]]

 [[ 74 255   1]
  [ 91 255   0]
  [113 255   0]
  ...
  [ 20 255  34]
  [ 21 255  31]
  [ 23 255  26]]

 ...

 [[  0 255  37]
  [179 255  37]
  [179 255  37]
  ..

[[[ 38 255   4]
  [ 39 255   4]
  [ 44 255   4]
  ...
  [179 255   6]
  [166 255   5]
  [167 255   4]]

 [[ 45 255   3]
  [ 46 255   3]
  [ 50 255   3]
  ...
  [  2 255   8]
  [  5 255   7]
  [ 14 255   7]]

 [[ 52 255   3]
  [ 51 255   3]
  [ 54 255   2]
  ...
  [ 10 255  13]
  [ 14 255  12]
  [ 19 255  10]]

 ...

 [[165 255   0]
  [103 255   0]
  [ 82 255   0]
  ...
  [ 97 255   3]
  [ 97 255   3]
  [ 98 255   2]]

 [[115 255   0]
  [101 255   0]
  [ 96 255   0]
  ...
  [ 93 255   2]
  [ 93 255   2]
  [ 94 255   1]]

 [[103 255   0]
  [ 98 255   0]
  [ 95 255   0]
  ...
  [ 93 255   2]
  [ 94 255   1]
  [ 96 255   1]]]
[[[ 37 255   4]
  [ 36 255   4]
  [ 38 255   3]
  ...
  [  5 255   6]
  [173 255   4]
  [175 255   3]]

 [[ 44 255   3]
  [ 43 255   3]
  [ 45 255   2]
  ...
  [  6 255   9]
  [  8 255   7]
  [ 15 255   7]]

 [[ 52 255   2]
  [ 50 255   2]
  [ 52 255   2]
  ...
  [ 11 255  13]
  [ 14 255  12]
  [ 17 255   9]]

 ...

 [[ 92 255   0]
  [ 87 255   0]
  [ 81 255   0]
  ..

[[[ 40 255   5]
  [ 38 255   5]
  [ 40 255   4]
  ...
  [ 14 255   3]
  [ 22 255   0]
  [ 95 255   3]]

 [[ 50 255   5]
  [ 46 255   4]
  [ 49 255   3]
  ...
  [174 255   5]
  [  6 255   3]
  [ 33 255   3]]

 [[ 62 255   4]
  [ 57 255   3]
  [ 62 255   3]
  ...
  [178 255   8]
  [ 15 255  10]
  [ 25 255   8]]

 ...

 [[133 255   0]
  [129 255   0]
  [122 255   0]
  ...
  [ 87 255   4]
  [ 86 255   4]
  [ 86 255   3]]

 [[115 255   0]
  [107 255   0]
  [104 255   0]
  ...
  [ 86 255   3]
  [ 86 255   3]
  [ 86 255   2]]

 [[100 255   0]
  [ 98 255   0]
  [ 97 255   0]
  ...
  [ 87 255   3]
  [ 87 255   2]
  [ 87 255   2]]]
[[[ 41 255   5]
  [ 41 255   6]
  [ 45 255   5]
  ...
  [  2 255   3]
  [144 255   1]
  [108 255   2]]

 [[ 52 255   5]
  [ 50 255   5]
  [ 53 255   5]
  ...
  [172 255   5]
  [179 255   3]
  [ 26 255   4]]

 [[ 59 255   5]
  [ 57 255   5]
  [ 60 255   4]
  ...
  [174 255   8]
  [ 10 255   9]
  [ 21 255   7]]

 ...

 [[103 255   0]
  [ 99 255   0]
  [ 98 255   0]
  ..

...

[[[ 41 255   1]
  [ 39 255   0]
  [ 67 255   0]
  ...
  [ 17 255   3]
  [ 18 255   1]
  [ 42 255   2]]

 [[ 58 255   1]
  [ 66 255   0]
  [ 90 255   0]
  ...
  [ 17 255   6]
  [ 25 255   5]
  [ 36 255   6]]

 [[ 68 255   1]
  [ 74 255   1]
  [ 90 255   0]
  ...
  [ 21 255  12]
  [ 26 255  10]
  [ 34 255   8]]

 ...

 [[154 255 182]
  [153 255 197]
  [152 255 208]
  ...
  [ 15 255   0]
  [ 23 255   0]
  [ 32 255   0]]

 [[153 255 147]
  [152 255 172]
  [152 255 197]
  ...
  [ 15 255   0]
  [ 25 255   0]
  [ 42 255   0]]

 [[154 255  58]
  [153 255  85]
  [152 255 155]
  ...
  [ 14 255   0]
  [ 22 255   0]
  [ 35 255   0]]]
[[[ 32 255   1]
  [ 30 255   1]
  [ 35 255   0]
  ...
  [ 27 255   2]
  [ 41 255   1]
  [ 48 255   3]]

 [[ 45 255   1]
  [ 43 255   0]
  [ 57 255   0]
  ...
  [ 22 255   5]
  [ 30 255   4]
  [ 38 255   6]]

 [[ 59 255   1]
  [ 58 255   0]
  [ 78 255   0]
  ...
  [ 23 255  10]
  [ 28 255   9]
  [ 35 255   8]]

 ...

 [[150 255 184]
  [149 255 199]
  [149 255 207]
  ..

[[[ 33 255   3]
  [ 35 255   4]
  [ 41 255   3]
  ...
  [ 15 255   4]
  [ 18 255   2]
  [ 39 255   5]]

 [[ 41 255   3]
  [ 41 255   3]
  [ 46 255   3]
  ...
  [  2 255   5]
  [ 16 255   4]
  [ 33 255  12]]

 [[ 50 255   2]
  [ 46 255   2]
  [ 51 255   2]
  ...
  [  2 255   7]
  [ 21 255  11]
  [ 31 255  17]]

 ...

 [[165 255 137]
  [167 255 139]
  [168 255 140]
  ...
  [175 255   0]
  [173 255   0]
  [176 255   0]]

 [[161 255 135]
  [162 255 136]
  [163 255 136]
  ...
  [ 12 255   0]
  [  9 255   0]
  [  6 255   0]]

 [[154 255 127]
  [155 255 127]
  [155 255 126]
  ...
  [178 255   0]
  [177 255   0]
  [177 255   0]]]
[[[ 30 255   5]
  [ 29 255   5]
  [ 29 255   4]
  ...
  [ 18 255   4]
  [ 26 255   2]
  [ 49 255   4]]

 [[ 36 255   4]
  [ 33 255   4]
  [ 33 255   3]
  ...
  [  7 255   5]
  [ 20 255   4]
  [ 36 255  10]]

 [[ 43 255   3]
  [ 37 255   3]
  [ 37 255   2]
  ...
  [  6 255   7]
  [ 23 255  12]
  [ 32 255  15]]

 ...

 [[177 255 111]
  [177 255 115]
  [177 255 118]
  ..

[[[ 33 255   3]
  [ 32 255   3]
  [ 33 255   2]
  ...
  [144 255   6]
  [135 255   9]
  [130 255  10]]

 [[ 41 255   3]
  [ 38 255   2]
  [ 40 255   2]
  ...
  [129 255  18]
  [132 255  15]
  [132 255  12]]

 [[ 52 255   2]
  [ 46 255   1]
  [ 49 255   1]
  ...
  [127 255  26]
  [130 255  22]
  [141 255   8]]

 ...

 [[137 255   0]
  [160 255   0]
  [169 255   1]
  ...
  [  1 255   0]
  [  4 255   1]
  [  7 255   1]]

 [[175 255   0]
  [  0 255   1]
  [  3 255   2]
  ...
  [176 255   0]
  [  0 255   1]
  [  3 255   1]]

 [[  0 255   0]
  [  2 255   1]
  [  4 255   2]
  ...
  [177 255   0]
  [  0 255   0]
  [  3 255   0]]]
[[[ 39 255   4]
  [ 37 255   4]
  [ 38 255   3]
  ...
  [128 255  10]
  [129 255   9]
  [125 255   9]]

 [[ 46 255   4]
  [ 44 255   3]
  [ 46 255   2]
  ...
  [124 255  29]
  [126 255  20]
  [127 255  11]]

 [[ 54 255   3]
  [ 50 255   2]
  [ 53 255   2]
  ...
  [123 255  35]
  [126 255  26]
  [139 255   6]]

 ...

 [[114 255   0]
  [122 255   0]
  [140 255   0]
  ..

[[[ 60 255   1]
  [ 53 255   1]
  [ 58 255   1]
  ...
  [  4 255   7]
  [167 255   5]
  [164 255   4]]

 [[ 70 255   2]
  [ 65 255   1]
  [ 71 255   1]
  ...
  [  6 255  10]
  [  3 255   9]
  [ 10 255   9]]

 [[ 71 255   2]
  [ 70 255   2]
  [ 75 255   1]
  ...
  [ 12 255  16]
  [ 14 255  17]
  [ 16 255  16]]

 ...

 [[119 255   0]
  [115 255   1]
  [115 255   1]
  ...
  [ 77 255   2]
  [ 76 255   2]
  [ 74 255   1]]

 [[128 255   0]
  [120 255   0]
  [118 255   0]
  ...
  [ 72 255   1]
  [ 70 255   1]
  [ 69 255   1]]

 [[130 255   0]
  [124 255   0]
  [125 255   0]
  ...
  [ 75 255   1]
  [ 73 255   1]
  [ 72 255   0]]]
[[[ 50 255   1]
  [ 46 255   1]
  [ 55 255   1]
  ...
  [ 14 255  12]
  [  6 255   8]
  [ 11 255   8]]

 [[ 67 255   1]
  [ 66 255   1]
  [ 75 255   1]
  ...
  [ 14 255  18]
  [ 13 255  15]
  [ 17 255  17]]

 [[ 71 255   2]
  [ 72 255   1]
  [ 78 255   1]
  ...
  [ 16 255  25]
  [ 16 255  24]
  [ 17 255  23]]

 ...

 [[105 255   0]
  [106 255   1]
  [106 255   1]
  ..

[[[ 18 255   4]
  [ 21 255   3]
  [ 23 255   2]
  ...
  [119 255  89]
  [119 255  71]
  [118 255  67]]

 [[ 20 255   2]
  [ 18 255   1]
  [  8 255   0]
  ...
  [119 255 104]
  [119 255  86]
  [119 255  78]]

 [[ 15 255   0]
  [  3 255   0]
  [149 255   0]
  ...
  [121 255 100]
  [121 255  84]
  [120 255  79]]

 ...

 [[ 92 255   3]
  [ 92 255   4]
  [ 91 255   5]
  ...
  [ 58 255   1]
  [ 50 255   1]
  [ 39 255   1]]

 [[ 92 255   2]
  [ 91 255   2]
  [ 90 255   3]
  ...
  [ 54 255   1]
  [ 43 255   1]
  [ 35 255   1]]

 [[ 92 255   1]
  [ 91 255   2]
  [ 90 255   2]
  ...
  [ 60 255   1]
  [ 44 255   0]
  [ 33 255   0]]]
[[[ 30 255   4]
  [ 31 255   4]
  [ 36 255   3]
  ...
  [ 10 255  10]
  [158 255   5]
  [ 20 255   0]]

 [[ 37 255   2]
  [ 36 255   2]
  [ 42 255   2]
  ...
  [ 15 255  19]
  [ 16 255  13]
  [ 27 255   8]]

 [[ 47 255   1]
  [ 43 255   1]
  [ 49 255   1]
  ...
  [ 20 255  38]
  [ 22 255  32]
  [ 25 255  16]]

 ...

 [[ 91 255   2]
  [ 91 255   3]
  [ 90 255   4]
  ..